In [30]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from PIL import Image
import scipy.stats as st
import pymc as pm
import arviz as az
import scipy.linalg as la
import seaborn as sns
import sqlite3
az.style.use('arviz-darkgrid')

In [5]:
bang = pd.read_csv('/Users/alseekford/6040 (Bayes)/HW4/bangladesh.csv')
bang

,woman,district,use.contraception,living.children,age.centered,urban
0,1,1,0,4,18.4400,1
1,2,1,0,1,-5.5599,1
2,3,1,0,3,1.4400,1
3,4,1,0,4,8.4400,1
4,5,1,0,1,-13.5590,1
...,...,...,...,...,...,...
1929,1930,61,0,4,14.4400,0
1930,1931,61,0,3,-4.5599,0
1931,1932,61,0,4,14.4400,0
1932,1933,61,0,1,-13.5600,0


In [10]:
bball = pd.read_csv('Use_This.csv')
bball

,Unnamed: 0,game_id,player_id,starter,mp,did_not_play,is_inactive,fantasy_points,player_name,position,team,opponent,teamscore,opponentscore,date,avg_pts
0,5038,202101200TOR,achiupr01,0,14:24,0,0,12.8,Precious Achiuwa,F,TOR,MIA,102,111,120,12.800000
1,5560,202101220TOR,achiupr01,0,11:32,0,0,10.7,Precious Achiuwa,F,TOR,MIA,101,81,122,12.800000
2,14277,202102240MIA,achiupr01,0,6:00,0,0,3.4,Precious Achiuwa,F,TOR,MIA,108,116,224,11.750000
3,42496,202110200TOR,achiupr01,1,18:30,0,0,18.4,Precious Achiuwa,F,TOR,WAS,83,98,1020,8.966667
4,42647,202110220BOS,achiupr01,1,24:50,0,0,35.0,Precious Achiuwa,F,TOR,BOS,115,83,1022,11.325000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30829,53979,202112200LAC,zubaciv01,1,27:21,0,0,31.9,Ivica Zubac,C,LAC,SAS,92,116,1220,20.970000
30830,54290,202112220SAC,zubaciv01,1,26:39,0,0,22.2,Ivica Zubac,C,LAC,SAC,105,89,1222,23.180000
30831,54770,202112260LAC,zubaciv01,1,27:22,0,0,36.2,Ivica Zubac,C,LAC,DEN,100,103,1226,22.760000
30832,54968,202112270LAC,zubaciv01,1,21:45,0,0,19.3,Ivica Zubac,C,LAC,BRK,108,124,1227,24.130000


In [15]:
bball.team.unique()

array(['TOR', 'MEM', 'MIA', 'BRK', 'NOP', 'MIL', 'CLE', 'SAS', 'LAL',
       'ORL', 'CHI', 'HOU', 'WAS', 'PHO', 'UTA', 'NYK', 'CHO', 'SAC',
       'DEN', 'PHI', 'LAC', 'OKC', 'MIN', 'GSW', 'DET', 'IND', 'POR',
       'ATL', 'BOS', 'DAL'], dtype=object)

## Creating new variables

In [17]:
# Team rank for 20-2021 season - two teams for each val (bc seperated by E & W conferences)

# Fx to map the values
def set_ranks(row_number, assigned_value):
    return assigned_value[row_number]
 
# Create the dictionary
ranks ={'PHI':1,
        'BRK':2,
        'MIL':3,
        'NYK':4,
        'ATL':5,
        'MIA':6,
        'BOS':7,
        'WAS':8,
        'IND':9,
        'CHO':10,
        'CHI':11,
        'TOR':12, 
        'CLE':13,
        'ORL':14,
        'DET':15,
        
        'UTA':1,
        'PHO':2,
        'DEN':3,
        'LAC':4,
        'DAL':5,
        'POR':6,
        'LAL':7,
        'MEM':8, 
        'GSW':9,
        'SAS':10,
        'NOP':11, 
        'SAC':12,
        'MIN':13,
        'OKC':14,
        'HOU':15
        }
 
# Add a new col
bball['team_rank'] = bball['team'].apply(set_ranks, args =(ranks, ))

In [25]:
# Conference
# 0 for East, 1 for West

bball['conference'] = [0 if x =='PHI' or x =='BRK' or x =='MIL' or x =='NYK' or x =='ATL' or x =='MIA' or x =='BOS' or x =='WAS' or x =='IND' or x =='CHO' or x =='CHI' or x =='TOR' or x =='CLE' or x =='ORL' or x =='DET' else 1 for x in bball['team']]

In [60]:
# Change var name for easier join

bball = bball.rename(columns={'player_name':'NAME'})

In [53]:
# Import box stats data (source: https://www.nbastuffer.com/2020-2021-nba-player-stats/) (20-21 season)

box_stats = pd.read_excel('20-21BoxMetrics.xlsx', header=1).reset_index()

In [54]:
box_stats = box_stats.drop(columns='RANK')

In [48]:
pd.set_option('display.max_columns', 500)
box_stats.columns

Index(['RANK', 'FULL NAME', 'TEAM', 'POS', 'AGE', 'GP', 'MPG',
       'MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor',
       'USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor',
       'TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions',
       'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%',
       'eFG%Effective Shooting PercentageWith eFG%, three-point shots made are worth 50% more than two-point shots made. eFG% Formula=(FGM+ (0.5 x 3PM))/FGA',
       'TS%True Shooting PercentageTrue shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.',
       'PPGPointsPoints per game.', 'RPGReboundsRebounds per game.',
       'TRB%Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the p

In [55]:
box_stats = box_stats.rename(columns={'FULL NAME':'NAME',
                                      'index':'RANK', 
                                      'MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor':'MIN%',
                                      'USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor':'USG%',
                                       'TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions':'TO%',
                                       'eFG%Effective Shooting PercentageWith eFG%, three-point shots made are worth 50% more than two-point shots made. eFG% Formula=(FGM+ (0.5 x 3PM))/FGA':'eFG%',
                                       'TS%True Shooting PercentageTrue shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.':'TS%',
                                       'PPGPointsPoints per game.':'PPG', 
                                       'RPGReboundsRebounds per game.':'RPG',
                                       'TRB%Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the player while the player is on the court.':'TRB%',
                                       'APGAssistsAssists per game.':'APG',
                                       'AST%Assist PercentageAssist percentage is an estimated percentage of teammate field goals a player assisted while the player is on the court':'AST%',
                                       'SPGStealsSteals per game.':'SPG', 
                                       'BPGBlocksBlocks per game.':'BPG',
                                       'TOPGTurnoversTurnovers per game.':'TOPG',
                                       'VIVersatility IndexVersatility index is a metric that measures a player’s ability to produce in points, assists, and rebounds. The average player will score around a five on the index, while top players score above 10':'VI',
                                       'ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.':'ORTG',
                                       'DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.':'DRTG'})

#### Box Stats Variable Descriptions:  

    - RANK: player rank
    - MIN%: Minutes PercentagePercentage of team minutes used by a player while he was on the floor
    - USG%: Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor
    - TO%: Turnover RateA metric that estimates the number of turnovers a player commits per 100 possession
    - eFG%: Effective Shooting PercentageWith eFG%, three-point shots made are worth 50% more than two-point shots made. eFG% Formula=(FGM+ (0.5 x 3PM))/FGA
    - TS%: True Shooting PercentageTrue shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws
    - PPG: PointsPoints per game
    - RPG: ReboundsRebounds per game
    - TRB%: Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the player while the player is on the court
    - APG: AssistsAssists per game
    - AST%: Assist PercentageAssist percentage is an estimated percentage of teammate field goals a player assisted while the player is on the court
    - SPG: StealsSteals per game
    - BPG: BlocksBlocks per game
    - TOPG: TurnoversTurnovers per game
    - VI: Versatility Index = Versatility index is a metric that measures a player’s ability to produce in points, assists, and rebounds. The average player will score around a five on the index, while top players score above 10
    - ORTG: Offensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions
    - DRTG: Defensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court

In [56]:
# Make teams all caps for an easier join

box_stats['TEAM'] = box_stats['TEAM'].str.upper()

In [57]:
box_stats

,RANK,NAME,TEAM,POS,AGE,GP,MPG,MIN%,USG%,TO%,FTA,FT%,2PA,2P%,3PA,3P%,eFG%,TS%,PPG,RPG,TRB%,APG,AST%,SPG,BPG,TOPG,VI,ORTG,DRTG
0,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.750,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6
1,1,Bryn Forbes,MIL,G,28.00,20,13.7,28.5,19.2,5.7,8,0.750,23,0.565,89,0.371,0.558,0.567,6.6,1.4,5.3,0.3,3.1,0.10,0.10,0.35,4.1,110.1,110.5
2,2,Jrue Holiday,MIL,G,31.11,23,39.6,82.6,22.1,11.9,56,0.714,247,0.466,142,0.303,0.461,0.482,17.3,5.7,7.8,8.7,31.4,1.65,0.43,2.43,9.6,108.9,108.4
3,3,Brook Lopez,MIL,C,33.30,23,29.0,60.4,16.8,9.6,43,0.860,145,0.662,72,0.319,0.601,0.632,13.0,5.9,10.9,0.3,1.3,0.74,1.48,1.09,3.9,122.1,106.8
4,4,Khris Middleton,MIL,F,29.94,23,40.1,83.5,26.4,13.0,97,0.887,277,0.498,175,0.343,0.504,0.548,23.6,7.6,10.1,5.1,19.6,1.52,0.17,3.22,9.7,107.8,106.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,234,Anthony Tolliver,PHI,F,36.14,1,1.7,3.5,0.0,NaN,0,0.000,0,0.000,0,0.000,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,NaN,NaN
235,235,Rayjon Tucker,PHI,G,23.82,1,1.7,3.5,22.3,0.0,2,0.500,0,0.000,0,0.000,NaN,0.568,1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,NaN,NaN
236,236,Justise Winslow,MEM,F-G,25.32,1,2.8,5.8,0.0,NaN,0,0.000,0,0.000,0,0.000,NaN,NaN,0.0,1.0,19.7,1.0,38.8,0.00,0.00,0.00,0.0,NaN,NaN
237,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.500,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6


#### NEW DATAFRAME!

In [68]:
# Join the two dataframes on the player name

stats = box_stats.merge(bball, on='NAME', how='inner')

,RANK,NAME,TEAM,POS,AGE,GP,MPG,MIN%,USG%,TO%,FTA,FT%,2PA,2P%,3PA,3P%,eFG%,TS%,PPG,RPG,TRB%,APG,AST%,SPG,BPG,TOPG,VI,ORTG,DRTG,Unnamed: 0,game_id,player_id,starter,mp,did_not_play,is_inactive,fantasy_points,position,team,opponent,teamscore,opponentscore,date,avg_pts,team_rank,conference
0,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,265,202101010MIL,connapa01,0,15:33,0,0,20.5,SG,MIL,CHI,126,96,101,20.500000,3,0
1,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,2438,202101090MIL,connapa01,0,10:53,0,0,10.8,SG,MIL,CLE,100,90,109,20.500000,3,0
2,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,2974,202101110ORL,connapa01,0,18:12,0,0,19.8,SG,MIL,ORL,121,99,111,15.650000,3,0
3,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,3370,202101130DET,connapa01,0,15:43,0,0,16.1,SG,MIL,DET,110,101,113,17.033333,3,0
4,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,3849,202101150MIL,connapa01,0,21:14,0,0,18.7,SG,MIL,DAL,112,109,115,16.800000,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16185,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,48731,202111200WAS,winstca01,0,0:00,1,0,0.0,PG,WAS,MIA,103,100,1120,1.080000,8,0
16186,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,49127,202111220WAS,winstca01,0,0:00,1,0,0.0,PG,WAS,CHO,103,109,1122,1.080000,8,0
16187,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,50801,202112010WAS,winstca01,0,0:00,1,0,0.0,PG,WAS,MIN,115,107,1201,1.080000,8,0
16188,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,51429,202112050TOR,winstca01,0,3:53,0,0,1.0,PG,WAS,TOR,90,102,1205,0.810000,8,0


In [72]:
stats = stats.drop(columns=['Unnamed: 0', 'team'])

In [74]:
stats

,RANK,NAME,TEAM,POS,AGE,GP,MPG,MIN%,USG%,TO%,FTA,FT%,2PA,2P%,3PA,3P%,eFG%,TS%,PPG,RPG,TRB%,APG,AST%,SPG,BPG,TOPG,VI,ORTG,DRTG,game_id,player_id,starter,mp,did_not_play,is_inactive,fantasy_points,position,opponent,teamscore,opponentscore,date,avg_pts,team_rank,conference
0,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,202101010MIL,connapa01,0,15:33,0,0,20.5,SG,CHI,126,96,101,20.500000,3,0
1,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,202101090MIL,connapa01,0,10:53,0,0,10.8,SG,CLE,100,90,109,20.500000,3,0
2,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,202101110ORL,connapa01,0,18:12,0,0,19.8,SG,ORL,121,99,111,15.650000,3,0
3,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,202101130DET,connapa01,0,15:43,0,0,16.1,SG,DET,110,101,113,17.033333,3,0
4,0,Pat Connaughton,MIL,G,28.54,23,23.7,49.4,10.7,8.8,13,0.846,24,0.75,95,0.389,0.618,0.633,6.9,4.4,9.9,0.9,4.8,0.43,0.26,0.52,5.1,129.0,107.6,202101150MIL,connapa01,0,21:14,0,0,18.7,SG,DAL,112,109,115,16.800000,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16185,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,202111200WAS,winstca01,0,0:00,1,0,0.0,PG,MIA,103,100,1120,1.080000,8,0
16186,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,202111220WAS,winstca01,0,0:00,1,0,0.0,PG,CHO,103,109,1122,1.080000,8,0
16187,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,202112010WAS,winstca01,0,0:00,1,0,0.0,PG,MIN,115,107,1201,1.080000,8,0
16188,237,Cassius Winston,WAS,G,23.39,1,5.5,11.5,38.0,40.0,0,0.000,2,0.50,1,0.000,0.333,0.333,2.0,2.0,19.8,1.0,27.4,0.00,0.00,2.00,11.5,61.3,98.6,202112050TOR,winstca01,0,3:53,0,0,1.0,PG,TOR,90,102,1205,0.810000,8,0


In [76]:
# Write to csv
stats.to_csv('player_game_stats.csv')